# Ollama + OpenAI + Python

## 1. Tukuyin ang pangalan ng modelo

Kung gumamit ka ng ibang modelo bukod sa "phi3:mini", palitan ang halaga sa cell sa ibaba. Ang variable na iyon ay gagamitin sa code sa buong notebook.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. I-setup ang Open AI client

Karaniwan, ginagamit ang OpenAI client kasama ng OpenAI.com o Azure OpenAI upang makipag-ugnayan sa malalaking modelo ng wika.  
Gayunpaman, maaari rin itong gamitin kasama ng Ollama, dahil nagbibigay ang Ollama ng OpenAI-compatible na endpoint sa "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Bumuo ng chat completion

Ngayon, maaari nating gamitin ang OpenAI SDK upang makabuo ng sagot para sa isang pag-uusap. Ang kahilingang ito ay dapat lumikha ng isang haiku tungkol sa mga pusa:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Pagbuo ng Prompt

Ang unang mensahe na ipinapadala sa language model ay tinatawag na "system message" o "system prompt", at ito ang nagtatakda ng pangkalahatang instruksyon para sa model.  
Maaari kang magbigay ng sarili mong custom na system prompt upang gabayan ang language model na gumawa ng output sa ibang paraan.  
Baguhin ang `SYSTEM_MESSAGE` sa ibaba upang sumagot na parang paborito mong sikat na karakter mula sa pelikula o TV, o kumuha ng inspirasyon para sa iba pang system prompts mula sa [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

Kapag na-customize mo na ang system message, ibigay ang unang tanong ng user sa `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Ilang halimbawa ng sagot

Isa pang paraan upang gabayan ang isang language model ay ang pagbibigay ng "ilang halimbawa", isang serye ng mga tanong/sagot na nagpapakita kung paano ito dapat tumugon.

Ang halimbawa sa ibaba ay sinusubukang gawing parang isang teaching assistant ang language model sa pamamagitan ng pagbibigay ng ilang halimbawa ng mga tanong at sagot na maaaring ibigay ng isang TA, at pagkatapos ay tinatanong ang model ng isang tanong na maaaring itanong ng isang estudyante.

Subukan muna ito, at pagkatapos ay baguhin ang `SYSTEM_MESSAGE`, `EXAMPLES`, at `USER_MESSAGE` para sa isang bagong sitwasyon.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Retrieval Augmented Generation

Ang RAG (Retrieval Augmented Generation) ay isang teknik na ginagamit upang makakuha ng tamang sagot mula sa isang language model para sa isang partikular na domain. Ginagawa ito sa pamamagitan ng pagkuha muna ng kaugnay na impormasyon mula sa isang knowledge source at pagkatapos ay bumubuo ng sagot batay sa impormasyong nakuha.

Nagbigay kami ng isang lokal na CSV file na naglalaman ng datos tungkol sa hybrid cars. Ang code sa ibaba ay nagbabasa ng CSV file, naghahanap ng mga tugma sa tanong ng user, at pagkatapos ay bumubuo ng sagot batay sa impormasyong nahanap. Tandaan na mas matagal ang prosesong ito kumpara sa mga naunang halimbawa, dahil mas maraming datos ang ipinapadala sa model. Kung mapansin mong ang sagot ay hindi pa rin nakabatay sa datos, maaari mong subukan ang system engineering o gumamit ng ibang mga modelo. Sa pangkalahatan, mas epektibo ang RAG kapag ginagamit ang mas malalaking modelo o ang mga fine-tuned na bersyon ng SLMs.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, tandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na dulot ng paggamit ng pagsasaling ito.
